<a href="https://colab.research.google.com/github/emukans/en-lv-translator/blob/master/LV_EN_Transformer_translate_on_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the [Tensor2Tensor](https://github.com/tensorflow/tensor2tensor) Colab

Tensor2Tensor, or T2T for short, is a library of deep learning models and datasets designed to make deep learning more accessible and [accelerate ML research](https://research.googleblog.com/2017/06/accelerating-deep-learning-research.html). In this notebook we will see how to use this library for a translation task by exploring the necessary steps. We will see how to define a problem, generate the data, train the model and test the quality of it, and we will translate our sequences and we visualize the attention. We will also see how to download a pre-trained model.

In [0]:
#@title
# Copyright 2018 Google LLC.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
# Install deps
!pip install -q -U tensor2tensor

#1. Initialization


##1.1. Make some directories

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import tensorflow as tf
import os

DRIVE_DIR = '/content/drive/My Drive'
DATA_DIR = DRIVE_DIR + "/t2t/data" # This folder contain the data
TMP_DIR = DRIVE_DIR + "/t2t/tmp"
TRAIN_DIR = DRIVE_DIR + "/t2t/train" # This folder contain the model
EXPORT_DIR = DRIVE_DIR + "/t2t/export" # This folder contain the exported model for production
TRANSLATIONS_DIR = DRIVE_DIR + "/t2t/translation" # This folder contain  all translated sequence
EVENT_DIR = DRIVE_DIR + "/t2t/event" # Test the BLEU score
USR_DIR = DRIVE_DIR + "/t2t/user" # This folder contains our data that we want to add

tf.gfile.MakeDirs(DATA_DIR)
tf.gfile.MakeDirs(TMP_DIR)
tf.gfile.MakeDirs(TRAIN_DIR)
tf.gfile.MakeDirs(EXPORT_DIR)
tf.gfile.MakeDirs(TRANSLATIONS_DIR)
tf.gfile.MakeDirs(EVENT_DIR)
tf.gfile.MakeDirs(USR_DIR)

## 1.2. Prepare a problem

In [0]:
# !gcloud auth application-default login

In [0]:
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_encoder
from tensor2tensor.data_generators import text_problems
from tensor2tensor.data_generators import translate
from tensor2tensor.data_generators import wiki_lm
from tensor2tensor.utils import registry

FLAGS = tf.flags.FLAGS

# End-of-sentence marker.
EOS = text_encoder.EOS_ID


@registry.register_problem
class TranslateEnLv(translate.TranslateProblem):
  """Problem spec for En-Lv translation."""

  @property
  def approx_vocab_size(self):
    return 2**14

  @property
  def vocab_type(self):
    return text_problems.VocabType.SUBWORD

  # @property
  # def is_generate_per_split(self):
  #   return False

  # @property
  # def dataset_splits(self):
  #   # Since we are responsible for generating the dataset splits, we override
  #   # `Text2TextProblem.dataset_splits` to specify that we intend to keep
  #   # 80% data for training and 10% for evaluation and testing each.
  #   return [{
  #       "split": problem.DatasetSplit.TRAIN,
  #       "shards": 80,
  #   }, {
  #       "split": problem.DatasetSplit.EVAL,
  #       "shards": 10,
  #   }, {
  #       "split": problem.DatasetSplit.TEST,
  #       "shards": 10,
  #   }]

  def source_data_files(self, dataset_split):
    return [
      (
        'gs://translator-lv-en/annotations.tgz',
        ('annotations/annotations.en', 'annotations/annotations.lv')
      )
    ]

  def generate_samples(self, data_dir, tmp_dir, dataset_split):
    datasets = self.source_data_files(dataset_split)

    tag = "train" if dataset_split == problem.DatasetSplit.TRAIN else "dev"
    data_path = translate.compile_data(tmp_dir, datasets, f'{self.name}-compiled-{tag}')
    
    for example in text_problems.text2text_txt_iterator(data_path + ".lang1", data_path + ".lang2"):
      yield example

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## 1.3. Init parameters





In [0]:
# PROBLEM = "translate_enfr_wmt32k" # We chose a problem translation English to French with 32.768 vocabulary
# PROBLEM = "translate_enfr_wmt_small8k" # We chose a problem translation English to French with 32.768 vocabulary
PROBLEM = "translate_en_lv" # We chose a problem translation English to French with 32.768 vocabulary
MODEL = "transformer" # Our model
# HPARAMS = "transformer_big" # Hyperparameters for the model by default 
HPARAMS = "transformer_base" # Hyperparameters for the model by default 
                            # If you have a one gpu, use transformer_big_single_gpu

In [0]:
#Show all problems and models 

from tensor2tensor.utils import registry
from tensor2tensor import problems

problems.available() #Show all problems
registry.list_models() #Show all registered models

#or

#Command line
# !t2t-trainer --registry_help #Show all problems
# !t2t-trainer --problems_help #Show all models

[]

# 2. Data generation 

Generate the data (download the dataset and generate the data).

---

 You can choose between command line or code.

## 2.1. Generate with code

In [0]:
t2t_problem = problems.problem(PROBLEM)
t2t_problem.generate_data(DATA_DIR, TMP_DIR)

INFO:tensorflow:Generating vocab file: /content/drive/My Drive/t2t/data/vocab.translate_en_lv.16384.subwords


INFO:tensorflow:Generating vocab file: /content/drive/My Drive/t2t/data/vocab.translate_en_lv.16384.subwords


INFO:tensorflow:Generating vocab from: [('gs://translator-lv-en/annotations.tgz', ('annotations/annotations.en', 'annotations/annotations.lv'))]


INFO:tensorflow:Generating vocab from: [('gs://translator-lv-en/annotations.tgz', ('annotations/annotations.en', 'annotations/annotations.lv'))]


INFO:tensorflow:Downloading gs://translator-lv-en/annotations.tgz to /content/drive/My Drive/t2t/tmp/annotations.tgz


...

INFO:tensorflow:write: 400000


INFO:tensorflow:write: 500000


INFO:tensorflow:write: 500000


INFO:tensorflow:Data shuffled.


INFO:tensorflow:Data shuffled.


# 3. Train the model





##3.1. Init parameters

You can choose between command line or code.

---

 batch_size :  a great value of preference.

---
train_steps : research paper mentioned 300k steps with 8 gpu on big transformer. So if you have 1 gpu, you will need to train the model x8 more. (https://arxiv.org/abs/1706.03762 for more information).



In [0]:
# train_steps = 300000 # Total number of train steps for all Epochs
# train_steps = 100000 # Total number of train steps for all Epochs
train_steps = 50000 # Total number of train steps for all Epochs
eval_steps = 100 # Number of steps to perform for each evaluation
# batch_size = 4096
batch_size = 2048
save_checkpoints_steps = 2000
ALPHA = 0.1
schedule = "continuous_train_and_eval"

You can choose schedule :
 

*  train. Bad quality
*  continuous_train_and_eval (default)
*   train_and_eval



##3.2. Train with code
create_hparams : https://github.com/tensorflow/tensor2tensor/blob/28adf2690c551ef0f570d41bef2019d9c502ec7e/tensor2tensor/utils/hparams_lib.py#L42

---
Change hyper parameters :
https://github.com/tensorflow/tensor2tensor/blob/28adf2690c551ef0f570d41bef2019d9c502ec7e/tensor2tensor/models/transformer.py#L1627


In [0]:
from tensor2tensor.utils.trainer_lib import create_run_config, create_experiment
from tensor2tensor.utils.trainer_lib import create_hparams
from tensor2tensor.utils import registry
from tensor2tensor import models
from tensor2tensor import problems

# Init Hparams object from T2T Problem
hparams = create_hparams(HPARAMS)

# Make Changes to Hparams
hparams.batch_size = batch_size
hparams.learning_rate = ALPHA
#hparams.max_length = 256

# Can see all Hparams with code below
#print(json.loads(hparams.to_json())

create_run_config : https://github.com/tensorflow/tensor2tensor/blob/28adf2690c551ef0f570d41bef2019d9c502ec7e/tensor2tensor/utils/trainer_lib.py#L105

---


create_experiment : https://github.com/tensorflow/tensor2tensor/blob/28adf2690c551ef0f570d41bef2019d9c502ec7e/tensor2tensor/utils/trainer_lib.py#L611

In [0]:
RUN_CONFIG = create_run_config(
      model_dir=TRAIN_DIR,
      model_name=MODEL,
      save_checkpoints_steps= save_checkpoints_steps
)

tensorflow_exp_fn = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        model_name=MODEL,
        problem_name=PROBLEM,
        data_dir=DATA_DIR, 
        train_steps=train_steps, 
        eval_steps=eval_steps, 
        use_xla=True # For acceleration
    ) 

tensorflow_exp_fn.train_and_evaluate()

Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.


Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.


INFO:tensorflow:Configuring DataParallelism to replicate the model.


INFO:tensorflow:Configuring DataParallelism to replicate the model.


INFO:tensorflow:schedule=continuous_train_and_eval


...

INFO:tensorflow:loss = 2.4841735, step = 24000 (107.226 sec)


INFO:tensorflow:global_step/sec: 0.971972


INFO:tensorflow:global_step/sec: 0.971972


INFO:tensorflow:loss = 2.4533012, step = 24100 (54.815 sec)


INFO:tensorflow:loss = 2.4533012, step = 24100 (54.815 sec)


#4. See the BLEU score

In [0]:
# #INIT FILE FOR TRANSLATE

# SOURCE_TEST_TRANSLATE_DIR = TMP_DIR+"/dev/newstest2014-fren-src.en.sgm"
# REFERENCE_TEST_TRANSLATE_DIR = TMP_DIR+"/dev/newstest2014-fren-ref.en.sgm"
# BEAM_SIZE=1

##4.1. Translate all
https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/bin/t2t_translate_all.py

In [0]:
# !t2t-translate-all \
#   --source=$SOURCE_TEST_TRANSLATE_DIR \
#   --model_dir=$TRAIN_DIR \
#   --translations_dir=$TRANSLATIONS_DIR \
#   --data_dir=$DATA_DIR \
#   --problem=$PROBLEM \
#   --hparams_set=$HPARAMS \
#   --output_dir=$TRAIN_DIR \
#   --t2t_usr_dir=$USR_DIR \
#   --beam_size=$BEAM_SIZE \
#   --model=$MODEL

##4.2. Test the BLEU score
The BLEU score for all translations: https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/bin/t2t_bleu.py#L68


In [0]:
# !t2t-bleu \
#    --translations_dir=$TRANSLATIONS_DIR \
#    --model_dir=$TRAIN_DIR \
#    --data_dir=$DATA_DIR \
#    --problem=$PROBLEM \
#    --hparams_set=$HPARAMS \
#    --source=$SOURCE_TEST_TRANSLATE_DIR \
#    --reference=$REFERENCE_TEST_TRANSLATE_DIR \
#    --event_dir=$EVENT_DIR

#5. Prediction of sentence


##5.1. Predict with code

In [0]:
import tensorflow as tf

#After training the model, re-run the environment but run this code in first, then predict.

tfe = tf.contrib.eager
tfe.enable_eager_execution()
Modes = tf.estimator.ModeKeys

In [0]:
#Config

from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics
import numpy as np

enfr_problem = problems.problem(PROBLEM)

# Copy the vocab file locally so we can encode inputs and decode model outputs
vocab_name = "vocab.translate_en_lv.8192.subwords"
vocab_file = os.path.join(DATA_DIR, vocab_name)

# Get the encoders from the problem
encoders = enfr_problem.feature_encoders(DATA_DIR)

ckpt_path = tf.train.latest_checkpoint(os.path.join(TRAIN_DIR))
print(ckpt_path)

def translate(inputs):
  encoded_inputs = encode(inputs)
  with tfe.restore_variables_on_create(ckpt_path):
    model_output = translate_model.infer(encoded_inputs)["outputs"]
  return decode(model_output)

def encode(input_str, output_str=None):
  """Input str to features dict, ready for inference"""
  inputs = encoders["inputs"].encode(input_str) + [1]  # add EOS id
  batch_inputs = tf.reshape(inputs, [1, -1, 1])  # Make it 3D.
  return {"inputs": batch_inputs}

def decode(integers):
  """List of ints to str"""
  integers = list(np.squeeze(integers))
  if 1 in integers:
    integers = integers[:integers.index(1)]
  return encoders["inputs"].decode(np.squeeze(integers))

In [0]:
#Predict 

hparams = trainer_lib.create_hparams(HPARAMS, data_dir=DATA_DIR, problem_name=PROBLEM)
translate_model = registry.model(MODEL)(hparams, Modes.PREDICT)

inputs = "the animal didn't cross the river because it was too tired"
# ref = "l'animal n'a pas traversé la rue parcequ'il etait trop fatigué" ## this just a reference for evaluate the quality of the traduction
outputs = translate(inputs)

print("Inputs: %s" % inputs)
print("Outputs: %s" % outputs)

file_input = open(os.path.join(TRANSLATIONS_DIR, "outputs_lv.txt"),"a")
file_input.write(f'{train_steps}: input - "{inputs}", output - "{outputs}"\n')
file_input.close()

# file_output = open("reference.fr","w+")
# file_output.write(ref)
# file_output.close()

##5.3. Evaluate the BLEU Score
BLEU score for a sequence translation: https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/bin/t2t_bleu.py#L24

In [0]:
# !t2t-bleu \
#     --translation=outputs.fr \
#     --reference=reference.fr

#6. Attention visualization
We need to have a predicted sentence with code.

##6.1. Attention utils


In [0]:
# from tensor2tensor.visualization import attention
# from tensor2tensor.data_generators import text_encoder

# SIZE = 35

# def encode_eval(input_str, output_str):
#   inputs = tf.reshape(encoders["inputs"].encode(input_str) + [1], [1, -1, 1, 1])  # Make it 3D.
#   outputs = tf.reshape(encoders["inputs"].encode(output_str) + [1], [1, -1, 1, 1])  # Make it 3D.
#   return {"inputs": inputs, "targets": outputs}

# def get_att_mats():
#   enc_atts = []
#   dec_atts = []
#   encdec_atts = []

#   for i in range(hparams.num_hidden_layers):
#     enc_att = translate_model.attention_weights[
#       "transformer/body/encoder/layer_%i/self_attention/multihead_attention/dot_product_attention" % i][0]
#     dec_att = translate_model.attention_weights[
#       "transformer/body/decoder/layer_%i/self_attention/multihead_attention/dot_product_attention" % i][0]
#     encdec_att = translate_model.attention_weights[
#       "transformer/body/decoder/layer_%i/encdec_attention/multihead_attention/dot_product_attention" % i][0]
#     enc_atts.append(resize(enc_att))
#     dec_atts.append(resize(dec_att))
#     encdec_atts.append(resize(encdec_att))
#   return enc_atts, dec_atts, encdec_atts

# def resize(np_mat):
#   # Sum across heads
#   np_mat = np_mat[:, :SIZE, :SIZE]
#   row_sums = np.sum(np_mat, axis=0)
#   # Normalize
#   layer_mat = np_mat / row_sums[np.newaxis, :]
#   lsh = layer_mat.shape
#   # Add extra dim for viz code to work.
#   layer_mat = np.reshape(layer_mat, (1, lsh[0], lsh[1], lsh[2]))
#   return layer_mat

# def to_tokens(ids):
#   ids = np.squeeze(ids)
#   subtokenizer = hparams.problem_hparams.vocabulary['targets']
#   tokens = []
#   for _id in ids:
#     if _id == 0:
#       tokens.append('<PAD>')
#     elif _id == 1:
#       tokens.append('<EOS>')
#     elif _id == -1:
#       tokens.append('<NULL>')
#     else:
#         tokens.append(subtokenizer._subtoken_id_to_subtoken_string(_id))
#   return tokens

# def call_html():
#   import IPython
#   display(IPython.core.display.HTML('''
#         <script src="/static/components/requirejs/require.js"></script>
#         <script>
#           requirejs.config({
#             paths: {
#               base: '/static/base',
#               "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.8/d3.min",
#               jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
#             },
#           });
#         </script>
#         '''))

##6.2 Display Attention

In [0]:
# import numpy as np

# # Convert inputs and outputs to subwords

# inp_text = to_tokens(encoders["inputs"].encode(inputs))
# out_text = to_tokens(encoders["inputs"].encode(outputs))

# hparams = trainer_lib.create_hparams(HPARAMS, data_dir=DATA_DIR, problem_name=PROBLEM)

# # Run eval to collect attention weights
# example = encode_eval(inputs, outputs)
# with tfe.restore_variables_on_create(tf.train.latest_checkpoint(ckpt_path)):
#   translate_model.set_mode(Modes.EVAL)
#   translate_model(example)
# # Get normalized attention weights for each layer
# enc_atts, dec_atts, encdec_atts = get_att_mats()

# call_html()
# attention.show(inp_text, out_text, enc_atts, dec_atts, encdec_atts)

#7. Export the model
For more information: https://github.com/tensorflow/tensor2tensor/tree/master/tensor2tensor/serving

In [0]:
# #export Model
# !t2t-exporter \
#   --data_dir=$DATA_DIR \
#   --output_dir=$TRAIN_DIR \
#   --problem=$PROBLEM \
#   --model=$MODEL \
#   --hparams_set=$HPARAMS \
#   --decode_hparams="beam_size=1,alpha=$ALPHA" \
#   --export_dir=$EXPORT_DIR

#8.Load pretrained model from Google Storage
We use the pretrained model En-De translation.

##8.1. See existing content storaged

In [0]:
# print("checkpoint: ")
# !gsutil ls "gs://tensor2tensor-checkpoints/transformer_ende_test"

# print("data: ")
# !gsutil ls "gs://tensor2tensor-data"

##8.2. Init model

In [0]:
# PROBLEM_PRETRAINED = "translate_ende_wmt32k"
# MODEL_PRETRAINED = "transformer" 
# HPARAMS_PRETRAINED = "transformer_base"

##8.3. Load content from google storage

In [0]:
# import tensorflow as tf
# import os


# DATA_DIR_PRETRAINED = os.path.expanduser("/t2t/data_pretrained")
# CHECKPOINT_DIR_PRETRAINED = os.path.expanduser("/t2t/checkpoints_pretrained")

# tf.gfile.MakeDirs(DATA_DIR_PRETRAINED)
# tf.gfile.MakeDirs(CHECKPOINT_DIR_PRETRAINED)


# gs_data_dir = "gs://tensor2tensor-data/"
# vocab_name = "vocab.translate_ende_wmt32k.32768.subwords"
# vocab_file = os.path.join(gs_data_dir, vocab_name)

# gs_ckpt_dir = "gs://tensor2tensor-checkpoints/"
# ckpt_name = "transformer_ende_test"
# gs_ckpt = os.path.join(gs_ckpt_dir, ckpt_name)

# TRAIN_DIR_PRETRAINED = os.path.join(CHECKPOINT_DIR_PRETRAINED, ckpt_name)

# !gsutil cp {vocab_file} {DATA_DIR_PRETRAINED}
# !gsutil -q cp -R {gs_ckpt} {CHECKPOINT_DIR_PRETRAINED}

# CHECKPOINT_NAME_PRETRAINED = tf.train.latest_checkpoint(TRAIN_DIR_PRETRAINED) # for translate with code


##8.4. Translate

In [0]:
# !echo "the business of the house" > "inputs.en"
# !echo -e "das Geschäft des Hauses" > "reference.de"

# !t2t-decoder \
#   --data_dir=$DATA_DIR_PRETRAINED \
#   --problem=$PROBLEM_PRETRAINED  \
#   --model=$MODEL_PRETRAINED  \
#   --hparams_set=$HPARAMS_PRETRAINED \
#   --output_dir=$TRAIN_DIR_PRETRAINED  \
#   --decode_hparams="beam_size=1" \
#   --decode_from_file="inputs.en" \
#   --decode_to_file="outputs.de"

# # See the translations
# !cat outputs.de

# !t2t-bleu \
#     --translation=outputs.de \
#     --reference=reference.de

#9.  Add your dataset/problem
To add a new dataset/problem, subclass Problem and register it with @registry.register_problem. See TranslateEnfrWmt8k for an example: 
https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/data_generators/translate_enfr.py

---
Adding your own components: https://github.com/tensorflow/tensor2tensor#adding-your-own-components

---

See this example: https://github.com/tensorflow/tensor2tensor/tree/master/tensor2tensor/test_data/example_usr_dir

In [0]:
# from tensor2tensor.utils import registry

# @registry.register_problem
# class MyTranslateEnFr(translate_enfr.TranslateEnfrWmt8k):

#   def generator(self, data_dir, tmp_dir, train):
#    #your code